In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.0.207 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.2/78.2 GB disk)


In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [ ]:

from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


In [ ]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [ ]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
# settings
MODEL = "/content/gdrive/MyDrive/New Model1/best.pt"


In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
results = model.track(source="https://youtu.be/LNwODJXcvt4", conf=0.3, iou=0.5, show=True)

# model.fuse()
# Export the model
# model.export(format='openvino')  # creates 'yolov8n_openvino_model/'

# ov_model = YOLO('/content/gdrive/MyDrive/New Model/best_openvino_model/')

# Load the exported OpenVINO model
# ov_model = YOLO('yolov8n_openvino_model/')

# Run inference
# results = ov_model('https://ultralytics.com/images/bus.jpg')

In [ ]:
import openvino as ov
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
im = np.stack([frame])
im = im.transpose((0, 3, 1, 2))  # BHWC to BCHW, (n, 3, h, w)
im = im.astype(np.float32) / 255.0
im = np.ascontiguousarray(im)  # contiguous
im = ov.Tensor(array=im, shared_memory=True)
results = ov_model(im)
detections = Detections(
    xyxy=results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)
# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]
# annotate and display frame
frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)

%matplotlib inline
show_frame_in_notebook(frame, (16, 16))

In [ ]:

# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# 0 - cow
# 1 - elbow
# 2 - fjoint
# 3 - foot
# 4 - hand
# 5 - hjoint
# 6 - hoof
# 7 - knee
# 8 - neck
# 9 - tc

In [ ]:
SOURCE_VIDEO_PATH = '/content/gdrive/MyDrive/test_video/test_video_to_tarck_3.mp4'
TARGET_VIDEO_PATH = "/content/gdrive/MyDrive/test_video/detection_all_03.mp4"

In [ ]:
from tqdm.notebook import tqdm
import cv2  # Import OpenCV for drawing

# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create LineCounter instance
colors = [(255, 0, 0), (0,255,0), (0, 255, 255), (255, 255, 0), (255, 0, 255), (128, 0, 0), (0, 0, 128), (128, 0, 128), (0, 128, 128), (0, 128, 0)]

# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)

# open target video file

changes = []
count = -1
befor_frame = []

with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame in tqdm(generator, total=video_info.total_frames):
        count+=1
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame, iou=0.4)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # format custom labels
        for detection in results[0].boxes:
          # print(detection.xyxy.cpu().numpy(), "lllll", detection.cls.cpu().numpy()[0])
          box = detection.xyxy.cpu().numpy()
          class_id = int(detection.cls.cpu().numpy()[0])
          cv2.rectangle(frame, (int(box[0][0]), int(box[0][1])), (int(box[0][2]), int(box[0][3])), colors[class_id], 2)

        # labels = [
        #     f"#{tracker_id}"
        #     for _, confidence, class_id, tracker_id
        #     in detections
        # ]

        # updating line counter
        # annotate and display frame
        # for _, confidence, class_id, tracker_id in detections:
          # print(confidence)
        # # Calculate and annotate the center of each box

        # if(count % 10 == 0):
        #   box = detection.xyxy.cpu().numpy()
        #   class_id = int(detection.cls.cpu().numpy()[0])
        #   x_center = int((int(box[0][0]) + int(box[0][2])) / 2)
        #   y_center = int((int(box[0][1]) + int(box[0][3])) / 2)
        #   if(count == 0):
        #     befor_frame.append((count, class_id, y_center))
        #   else:
        #     for befor in befor_frame:
        #       if befor[1] == class_id:
            # Draw a point at the center

          # (0, 255, 0) is the color, 5 is the radius

        # line_annotator.annotate(frame=frame, line_counter=line_counter)
        # frame = box_annotator.annotate(frame=frame, detections=detections)
        # for d in detections:
        # image = cv2.rectangle(frame, detectio, end_point, color, thickness)
        sink.write_frame(frame)


  0%|          | 0/562 [00:00<?, ?it/s]


0: 480x800 1 Cow, 1 elbow, 1 fjoint, 2 foots, 1 hand, 1 hoof, 1 knee, 1 neck, 15.6ms
Speed: 3.9ms preprocess, 15.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 Cow, 1 elbow, 1 fjoint, 2 foots, 1 hand, 1 hoof, 1 knee, 1 neck, 15.5ms
Speed: 4.3ms preprocess, 15.5ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 Cow, 1 elbow, 1 fjoint, 2 foots, 2 hands, 1 hoof, 1 knee, 1 neck, 15.4ms
Speed: 2.7ms preprocess, 15.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 Cow, 1 elbow, 1 fjoint, 2 foots, 2 hands, 1 hoof, 1 knee, 2 necks, 15.6ms
Speed: 4.2ms preprocess, 15.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 Cow, 1 elbow, 1 fjoint, 2 foots, 2 hands, 1 hoof, 1 knee, 1 neck, 15.5ms
Speed: 3.5ms preprocess, 15.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 Cow, 1 elbow, 2 foots, 2 hands, 1 hoof, 1 knee, 1 neck, 15.5ms
Spee

In [ ]:
import math
import cv2
def extract_frames(video_path, output_directory, interval):
    # Load video
    video = cv2.VideoCapture(video_path)

    # Get frame rate of the video
    fps = video.get(cv2.CAP_PROP_FPS)

    # Calculate frame interval based on desired interval and video frame rate
    frame_interval = math.floor(fps * interval)

    # Initialize frame counter
    frame_count = 0
    count=0

    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        count+=1

        # Break the loop if no frame is returned
        if not ret:
            break

        # Save the frame to the output directory
        output_path = f"{output_directory}/frame_{count}.jpg"
        cv2.imwrite(output_path, frame)

        # Increment frame counter
        frame_count += frame_interval

        # Set the video pointer to the next frame based on frame_interval
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)

    # Release the video object
    video.release()

# Example usage
folder_path = "/content/gdrive/MyDrive/test_video/test_video_to_tarck_9.mp4"
output_directory = "/content/gdrive/MyDrive/test_video/extracted_9"
extract_frames(folder_path, output_directory, 1.5)

In [ ]:
video = cv2.VideoCapture("/content/gdrive/MyDrive/test_video/test_video_to_tarck_8.mp4")

    # Get frame rate of the video
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)

30.0
